In [106]:
#import libs
from neo4j import GraphDatabase
import pandas as pd

In [107]:
#conexão com o banco de dados
uri = "bolt://localhost:7687"
username = "luiz88"
password = "12345678"

In [123]:
df = pd.read_csv('steam.csv')

In [124]:
#abre a conexão
driver = GraphDatabase.driver(uri, auth=(username, password))

In [125]:
relations = [['developer','platforms','DEVELOPING_TO'],
             ['publisher','genres','DEVELOPING_TO'],
             ['publisher','steamspy_tags','DEVELOPING_TO'],]

In [126]:
def create_relationship(node_1, node_2, relation_type):
    ''''
    Função para criar relacionamento entre o nó game e os demais nós
    '''
    with driver.session() as session:
        node_2[1] = node_2[1].replace("'","")
        session.run(F"""MATCH (a:{node_1[0]} {{name: '{node_1[1]}'}}), 
                        (b:{node_2[0]} {{name: '{node_2[1]}'}})
                        CREATE (a)-[:{relation_type}_{node_2[0].upper()}]->(b)
                    """)

In [127]:
for rel in relations:
    qtd_columns = df[rel[1]].str.split(';', expand=True).shape[1]
    df_temp = pd.DataFrame()

    for round in range(0,qtd_columns):
        df_temp_value = df[rel[1]].str.split(';', expand=True)[round]
        df_temp_key = df[rel[0]]
        df_round = pd.concat([df_temp_key,df_temp_value], axis=1).rename(columns={round:'value'})
        df_temp = pd.concat([df_temp,df_round], axis=0).drop_duplicates().dropna()
        df_temp['relation'] = rel[-1]

    for event in zip(df_temp[rel[0]],df_temp['value'],df_temp['relation']):
        create_relationship([rel[0],event[0].replace("'","")], 
                            [rel[1],event[1].replace("'","")],
                            event[2])

'Travellers Tales'